0. Settings

In [0]:
!mkdir logs
!mkdir results
!mkdir data
!mkdir models

1. Download dataset

In [0]:
!pip install --upgrade gallery-dl
!pip install animeface
!gallery-dl https://danbooru.donmai.us/posts?tags=face

In [0]:
import glob
import os
import animeface
from PIL import Image

total_num_faces=0
for index,filename in enumerate(glob.glob('./gallery-dl/danbooru/face/*.*')):
  try:
    image=Image.open(filename)
    faces=animeface.detect(image)
  except Exception as e:
    print("Exception : {}".format(e))
  
  if len(faces)==0:
    continue
  
  fp=faces[0].face.pos
  coordinates=(fp.x,fp.y,fp.x+fp.width,fp.y+fp.height)
  cropped_image=image.crop(coordinates)
  cropped_mage=cropped_image.resize((64,64),Image.ANTIALIAS)
  total_num_faces+=1
  cropped_image.save("data/"+str(total_num_faces)+'.png')
  
print("total number of faces : {}".format(total_num_faces))


In [0]:
!zip -r data.zip data

2. Build models

In [0]:
import glob
import time
import numpy as np
import tensorflow as tf
from PIL import Image
import cv2

In [0]:
def build_generator():
    gen_model = tf.keras.Sequential()

    gen_model.add(tf.keras.layers.Dense(units=2048,input_shape=(100,)))
    gen_model.add(tf.keras.layers.ReLU())

    gen_model.add(tf.keras.layers.Dense(units=256 * 8 * 8))
    gen_model.add(tf.keras.layers.BatchNormalization())
    gen_model.add(tf.keras.layers.ReLU())
    gen_model.add(tf.keras.layers.Reshape((8, 8, 256), input_shape=(256 * 8 * 8,)))
    gen_model.add(tf.keras.layers.UpSampling2D(size=(2, 2)))

    gen_model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same'))
    gen_model.add(tf.keras.layers.ReLU())

    gen_model.add(tf.keras.layers.UpSampling2D(size=(2, 2)))

    gen_model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same'))
    gen_model.add(tf.keras.layers.ReLU())

    gen_model.add(tf.keras.layers.UpSampling2D(size=(2, 2)))

    gen_model.add(tf.keras.layers.Conv2D(3, (5, 5), padding='same'))
    gen_model.add(tf.keras.layers.Activation('tanh'))
    return gen_model


def build_discriminator():
    dis_model = tf.keras.Sequential()

    dis_model.add(tf.keras.layers.Conv2D(128, (5, 5),padding='same',input_shape=(64, 64, 3)))
    dis_model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    dis_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    dis_model.add(tf.keras.layers.Conv2D(256, (3, 3)))
    dis_model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    dis_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    dis_model.add(tf.keras.layers.Conv2D(512, (3, 3)))
    dis_model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    dis_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    dis_model.add(tf.keras.layers.Flatten())
    dis_model.add(tf.keras.layers.Dense(1024))
    dis_model.add(tf.keras.layers.LeakyReLU(alpha=0.2))

    dis_model.add(tf.keras.layers.Dense(1))
    dis_model.add(tf.keras.layers.Activation('sigmoid'))

    return dis_model


def build_adversarial_model(gen_model, dis_model):
    model = tf.keras.Sequential()
    model.add(gen_model)
    dis_model.trainable = False
    model.add(dis_model)
    return model

3. Train

In [0]:
def train(dataset_dir='data',batch_size=128,epochs=5000):
  start_time = time.time()
  dataset_path = dataset_dir+"/*.jpg"
  z_shape = 100
  dis_learning_rate = 0.005
  gen_learning_rate = 0.005
  dis_momentum = 0.5
  gen_momentum = 0.5
  dis_nesterov = True
  gen_nesterov = True

  dis_optimizer = tf.keras.optimizers.SGD(lr=dis_learning_rate, momentum=dis_momentum, nesterov=dis_nesterov)
  gen_optimizer = tf.keras.optimizers.SGD(lr=gen_learning_rate, momentum=gen_momentum, nesterov=gen_nesterov)

  # Load images
  all_images = []

  for index, filename in enumerate(glob.glob(dataset_path)):
    # print(filename)
    image=cv2.imread(filename)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(64,64))
    all_images.append(image)
  
  X = np.array(all_images)
  X = (X - 127.5) / 127.5
  X = X.astype(np.float32)

  print(X.shape)

  dis_model = build_discriminator()
  dis_model.compile(loss='binary_crossentropy', optimizer=dis_optimizer)

  gen_model = build_generator()
  gen_model.compile(loss='mse', optimizer=gen_optimizer)

  adversarial_model = build_adversarial_model(gen_model, dis_model)
  adversarial_model.compile(loss='binary_crossentropy', optimizer=gen_optimizer)

  tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/{}".format(time.time()), write_images=True, write_grads=True, write_graph=True)
  tensorboard.set_model(gen_model)
  tensorboard.set_model(dis_model)

  # train
  for epoch in range(epochs):
      print("--------------------------")
      print("Epoch:{}".format(epoch))

      dis_losses = []
      gen_losses = []

      num_batches = int(X.shape[0] / batch_size)

      print("Number of batches:{}".format(num_batches))

      for index in range(num_batches):
        print("Batch:{}".format(index),end=":")

        # train discriminator model
        z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
        generated_images = gen_model.predict_on_batch(z_noise)

        image_batch = X[index * batch_size:(index + 1) * batch_size]

        y_real = np.ones((batch_size, )) * 0.9
        y_fake = np.zeros((batch_size, )) * 0.1

        dis_model.trainable = True
        dis_loss_real = dis_model.train_on_batch(image_batch, y_real)
        dis_loss_fake = dis_model.train_on_batch(generated_images, y_fake)
        dis_model.trainable = False

        d_loss = (dis_loss_real+dis_loss_fake)/2
        print("d_loss:", d_loss,end=", ")

        
        # train generator model
        z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
        g_loss = adversarial_model.train_on_batch(z_noise, y_real)
        print("g_loss:", g_loss)

        dis_losses.append(d_loss)
        gen_losses.append(g_loss)
      

      # save images
      if epoch % 100 == 0:

        z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
        gen_images1 = gen_model.predict_on_batch(z_noise)

        for img in gen_images1[:2]:
          img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
          cv2.imwrite("results/result_{}.png".format(epoch),img)
          
          plt.imshow(img)
          plt.axis("off")
          plt.show()
          #plt.savefig("results/result_{}.png".format(epoch))
          #plt.close()

        print("Epoch:{}, dis_loss:{}".format(epoch, np.mean(dis_losses)))
        print("Epoch:{}, gen_loss: {}".format(epoch, np.mean(gen_losses)))

          
        #Save losses to Tensorboard after each epoch
        tf.summary.scalar('discriminator_loss', np.mean(gen_losses), step=epoch)
        tf.summary.scalar('generator_loss', np.mean(dis_losses), step=epoch)

    
  #Save models
  gen_model.save("generator_model.h5")
  dis_model.save("discriminator_model.h5")

  print("Time:", (time.time() - start_time))

In [33]:
if __name__ == '__main__':
  train(dataset_dir='gallery-dl/danbooru/face',epochs=1000)

d_loss: 0.17030542041175067, g_loss: 5.51571798324585
Batch:3:d_loss: 0.16721591888926923, g_loss: 5.573037147521973
--------------------------
Epoch:469
Number of batches:4
Batch:0:d_loss: 0.17011245910543948, g_loss: 5.48972225189209
Batch:1:d_loss: 0.1692319291178137, g_loss: 5.542644500732422
Batch:2:d_loss: 0.17022746475413442, g_loss: 5.525187969207764
Batch:3:d_loss: 0.16716387495398521, g_loss: 5.587530136108398
--------------------------
Epoch:470
Number of batches:4
Batch:0:d_loss: 0.17004983662627637, g_loss: 5.512739181518555
Batch:1:d_loss: 0.16915512236300856, g_loss: 5.562408447265625
Batch:2:d_loss: 0.17015613871626556, g_loss: 5.5403337478637695
Batch:3:d_loss: 0.16711325198411942, g_loss: 5.597954273223877
--------------------------
Epoch:471
Number of batches:4
Batch:0:d_loss: 0.16995878797024488, g_loss: 5.514322280883789
Batch:1:d_loss: 0.16909575555473566, g_loss: 5.565984725952148
Batch:2:d_loss: 0.17007479874882847, g_loss: 5.548093795776367
Batch:3:d_loss: 0.16